In [3028]:
import numpy as np
import pandas as pd

In [3029]:
space = pd.read_csv("D:/DataSets for ML/spaceship-titanic/train.csv")

In [3030]:
space.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3031]:
x = space.drop(columns='Transported')
y = space['Transported']

In [3032]:
x.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


In [3033]:
y.head()

0    False
1     True
2    False
3    False
4     True
Name: Transported, dtype: bool

#### 1. Drop Name

In [3034]:
x = x.drop(columns='Name')

In [3035]:
x.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0


#### 2. Splitting **PassengerId** into **GroupID** and **GroupMember**

In [3036]:
x[['GroupID','GroupMember']] = x['PassengerId'].str.split('_',expand=True)

x['GroupID'] = x['GroupID'].astype(int)
x['GroupMember'] = x['GroupMember'].astype(int)

In [3037]:
x = x.drop(columns='PassengerId')

In [3038]:
x['GroupSize'] = x.groupby('GroupID')['GroupID'].transform('count')

In [3039]:
x['IsAlone'] = x['GroupSize'] == 1

In [3040]:
x['IsAlone'] = x['IsAlone'].astype(int)

#### 3. Splitting **Cabin** into **CabinDeck**, **CabinNum**, **CabinSide**

In [3041]:
x[['CabinDeck','CabinNum','CabinSide']] = x['Cabin'].str.split('/',expand=True)

In [3042]:
x = x.drop(columns='Cabin')

In [3043]:
x['CabinNum'] = x['CabinNum'].fillna(-1).astype(int)

In [3044]:
x['CabinDeck'] = x['CabinDeck'].fillna('M')
x['CabinSide'] = x['CabinSide'].fillna('M')

In [3045]:
from sklearn.preprocessing import LabelEncoder

le_CabinDeck = LabelEncoder()
le_CabinSide = LabelEncoder()

x['CabinDeck'] = le_CabinDeck.fit_transform(x['CabinDeck'])
x['CabinSide'] = le_CabinSide.fit_transform(x['CabinSide'])

#### 4. **TotalSpend** = RoomService + FoodCourt + ShoppingMall + Spa + VRDeck 

In [3046]:
x['TotalSpend'] = x[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1)

In [3047]:
x = x.drop(columns=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])

#### 5. **VIP** from False/True to **IsVIP** to 0/1

In [3048]:
x['IsVIP'] = x['VIP'].map({False: 0, True: 1})

In [3049]:
x['IsVIP'] = x['IsVIP'].fillna(-1).astype(int)

In [3050]:
x = x.drop(columns='VIP')

#### 6. **CryoSleep** from False/True to **IsCryoSleep** to 0/1

In [3051]:
x['IsCryoSleep'] = x['CryoSleep'].map({False: 0, True: 1})
x['IsCryoSleep'] = x['IsCryoSleep'].fillna(-1).astype(int)

In [3052]:
x = x.drop(columns='CryoSleep')

#### 7. **Age** with missing values as -1

In [3053]:
x['Age'] = x['Age'].fillna(-1)

#### 8. HomePlanet and Destination

In [3054]:
x['HomePlanet'] = x['HomePlanet'].fillna("Missing")

In [3055]:
x['Destination'] = x['Destination'].fillna("Missing")

In [3056]:
le_HomePlanet = LabelEncoder()
le_Destination = LabelEncoder()

x['HomePlanet'] = le_HomePlanet.fit_transform(x['HomePlanet'])
x['Destination'] = le_Destination.fit_transform(x['Destination'])

#### 9. GroupId

In [3057]:
x = x.drop(columns='GroupID')

### Training The Models

In [3058]:
from sklearn.preprocessing import StandardScaler

numeric_cols = ['Age', 'GroupSize', 'CabinNum', 'TotalSpend']

scaler = StandardScaler()
x[numeric_cols] = scaler.fit_transform(x[numeric_cols])

In [3059]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter=2000)
LR.fit(x, y)

LogisticRegression(max_iter=2000)

In [3060]:
x_test = pd.read_csv("D:/DataSets for ML/spaceship-titanic/test.csv")

#### Now Processing of x_test

In [3061]:
x_test = x_test.drop(columns='Name')

In [3062]:
x_test[['GroupID','GroupMember']] = x_test['PassengerId'].str.split('_',expand=True)

x_test['GroupID'] = x_test['GroupID'].astype(int)
x_test['GroupMember'] = x_test['GroupMember'].astype(int)

In [3063]:
x_test = x_test.drop(columns='PassengerId')

In [3064]:
x_test['GroupSize'] = x_test.groupby('GroupID')['GroupID'].transform('count')

In [3065]:
x_test['IsAlone'] = x_test['GroupSize'] == 1

In [3066]:
x_test['IsAlone'] = x_test['IsAlone'].astype(int)

In [3067]:
x_test[['CabinDeck','CabinNum','CabinSide']] = x_test['Cabin'].str.split('/',expand=True)

In [3068]:
x_test = x_test.drop(columns='Cabin')

In [3069]:
x_test['CabinNum'] = x_test['CabinNum'].fillna(-1).astype(int)

In [3070]:
x_test['CabinDeck'] = x_test['CabinDeck'].fillna('M')
x_test['CabinSide'] = x_test['CabinSide'].fillna('M')

In [3071]:
x_test['CabinDeck'] = le_CabinDeck.transform(x_test['CabinDeck'])
x_test['CabinSide'] = le_CabinSide.transform(x_test['CabinSide'])

In [3072]:
x_test['TotalSpend'] = x_test[['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']].sum(axis=1)

In [3073]:
x_test = x_test.drop(columns=['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])

In [3074]:
x_test['IsVIP'] = x_test['VIP'].map({False: 0, True: 1})

In [3075]:
x_test['IsVIP'] = x_test['IsVIP'].fillna(-1).astype(int)

In [3076]:
x_test = x_test.drop(columns='VIP')

In [3077]:
x_test['IsCryoSleep'] = x_test['CryoSleep'].map({False: 0, True: 1})
x_test['IsCryoSleep'] = x_test['IsCryoSleep'].fillna(-1).astype(int)

In [3078]:
x_test = x_test.drop(columns='CryoSleep')

In [3079]:
x_test['Age'] = x_test['Age'].fillna(-1)

In [3080]:
x_test['HomePlanet'] = x_test['HomePlanet'].fillna("Missing")
x_test['Destination'] = x_test['Destination'].fillna("Missing")

In [3081]:
x_test['HomePlanet'] = le_HomePlanet.transform(x_test['HomePlanet'])
x_test['Destination'] = le_Destination.transform(x_test['Destination'])

In [3082]:
numeric_cols = ['Age', 'GroupSize', 'CabinNum', 'TotalSpend']

x_test[numeric_cols] = scaler.transform(x_test[numeric_cols])

In [3083]:
x_test = x_test.drop(columns='GroupID')

In [3084]:
x_test.sample(5)

,HomePlanet,Destination,Age,GroupMember,GroupSize,IsAlone,CabinDeck,CabinNum,CabinSide,TotalSpend,IsVIP,IsCryoSleep
3324,0,2,2.393595,1,-0.648735,1,5,1.586760,2,0.439238,0,0
3671,1,0,-0.148068,2,1.230665,0,2,-0.635563,1,0.691121,0,0
3703,0,3,-0.549383,1,-0.648735,1,6,1.396053,2,-0.257187,0,0
38,0,3,-1.485785,1,-0.022268,0,6,-1.108439,2,-0.514066,0,1
1979,0,3,-0.616269,1,-0.648735,1,6,0.222619,1,-0.225434,0,0


In [3085]:
y_pred_LR = LR.predict(x_test)

In [3086]:
y_pred_LR

array([ True, False,  True, ...,  True, False,  True])

In [3087]:
y_test = pd.read_csv("D:/DataSets for ML/spaceship-titanic/sample_submission.csv")

In [3088]:
passenger = y_test['PassengerId']

In [3089]:
y_test = y_test.drop(columns='PassengerId')

In [3090]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred_LR)

0.5803133037175591

In [3091]:
import pandas as pd

df = pd.DataFrame({
    'PassengerId': passenger.values,
    'Transported': y_pred_LR
})

df.to_csv("output.csv", index=False)
